In [1]:
import numpy as np
import pandas as pd
import GPy
from scipy.special import expit
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from scipy.spatial import distance
import statsmodels.api as sm
import warnings
import numpy as np
from scipy.optimize import differential_evolution
from scipy.stats import rankdata
from sklearn.cross_decomposition import CCA
import math
import copy
from math import log
from collections import Counter
import pickle
from sklearn.linear_model import Lasso
warnings.filterwarnings("ignore")

In [2]:
adjacency_matrix = np.load('adjacency_matrix.npy')

# Read beta back from the file
beta_star = np.load('beta_star.npy')
print("adjacency_matrix:", adjacency_matrix)
print("beta:", beta_star)

adjacency_matrix: [[0. 1. 0. 1. 1. 0. 1. 1. 1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 1. 0. 0. 1. 0. 0. 1. 1. 1. 0. 1. 0.]
 [0. 0. 0. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 0. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1.]
 [1. 0. 1. 1. 0. 1. 0. 1. 1. 0. 1. 0. 1. 0. 1.]
 [0. 0. 1. 1. 1. 0. 0. 1. 0. 1. 0. 1. 1. 0. 0.]
 [1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [1. 0. 0. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 1. 0. 1. 0. 0. 0. 0. 1. 0. 1. 1. 1. 1.]
 [0. 1. 1. 1. 0. 1. 0. 0. 1. 0. 1. 1. 0. 0. 1.]
 [0. 1. 1. 1. 1. 0. 0. 0. 0. 1. 0. 1. 1. 1. 1.]
 [0. 1. 1. 1. 0. 1. 0. 0. 1. 1. 1. 0. 1. 1. 1.]
 [0. 0. 1. 1. 1. 1. 1. 0. 1. 0. 1. 1. 0. 0. 0.]
 [0. 1. 1. 1. 0. 0. 0. 0. 1. 0. 1. 1. 0. 0. 0.]
 [0. 0. 1. 1. 1. 0. 0. 0. 1. 1. 1. 1. 0. 0. 0.]]
beta: [ 0.          0.          0.          0.          0.          0.
  0.          0.         11.92974738  0.          0.          0.
 -4.73132762  0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.        

In [3]:
K = 2
error_std = 1

In [4]:
def transform_treatment(adjacency_matrix, A, K):
    # Extract upper triangular part (excluding diagonal)
    upper_triangular = np.triu(adjacency_matrix, k=1)
    
    # Generate B based on the rules
    B = []
    for i in range(len(adjacency_matrix)):
        for j in range(len(adjacency_matrix)):
            if upper_triangular[i, j] == 1:
                B.append(K * (A[i] - 1) + A[j])
    
    B = np.array(B)
    
    # Create X
    # Step 1: Start with 1
    X = [1]
    
    # Step 2: Add one-hot encoding of A (length K for each element in A)
    for a in A:
        one_hot_a = np.eye(K)[a - 1]  # One-hot encoding
        X.extend(one_hot_a)
    
    # Step 3: Add one-hot encoding of B (length K^2 for each element in B)
    for b in B:
        one_hot_b = np.eye(K**2)[b - 1]  # One-hot encoding
        X.extend(one_hot_b)
    
    X = np.array(X)

    return X

In [5]:
def while_ete_bandit(beta_star, E, lamada):

    error_std = 1
    T = 1002
    L = 1
    K = 2

    X_exper = []
    
    for t in range(E):
        A = np.random.randint(1, K + 1, size=len(adjacency_matrix))
        X = transform_treatment(adjacency_matrix, A, K)
        X_exper.append(X)

    X_exper = np.array(X_exper)

    # Generate random error term
    error_exper = np.random.normal(0, error_std, E)
    
    # Compute Y = X @ beta_star + error_exper
    Y_t = X_exper @ beta_star + error_exper
    Y_t_exp = X_exper @ beta_star

    Y_t_exp = ete_bandit(T, K, adjacency_matrix, error_std, Y_t, X_exper, beta_star, Y_t_exp, lamada)

    return Y_t_exp

In [6]:
def ete_bandit(T, K, adjacency_matrix, error_std, Y_t, X_exper, beta_star, Y_t_exp, lamada):
    
    n_features = X_exper.shape[1]  # Number of features

    # Perform LASSO regression with lambda using Y_t_tran and X_t_tran
    lasso = Lasso(alpha=lamada, fit_intercept=False)
    lasso.fit(X_exper, Y_t)  # Fit the model to get beta estimates
    beta_esti = lasso.coef_  # Estimated beta values


    A = optimization(beta_esti, K, adjacency_matrix)

    X_max = transform_treatment(adjacency_matrix, A, K)

    Y_max_exp = beta_star @ X_max

    # Append Y_max_exp for the remaining time steps
    remaining_steps = T - len(Y_t)
    Y_t_exp = np.append(Y_t_exp, [Y_max_exp] * remaining_steps)

    return Y_t_exp

In [7]:
def optimization(beta_esti, K, adjacency_matrix):
    # Define the objective function
    # Define the objective function
    def objective(X_max):
        X_max = np.array(X_max)
        return -(beta_esti @ X_max)  # Negate for maximization
    
    # Initialize X_max with all zeros
    num_vars = len(adjacency_matrix)
    A = np.random.randint(1, K + 1, size=len(adjacency_matrix))
    
    X = transform_treatment(adjacency_matrix, A, K)
    
    epochs = 5
    for epoch in range(epochs):
        current_value = objective(X)

        # Greedy optimization within this epoch
        for i in range(num_vars):
            # Flip the value of X_max[i] (0 -> L, L -> 0)
            original_value = A[i]

            
            # only useful for K = 2
            A[i] = 1 if original_value == 2 else 2
            
            X = transform_treatment(adjacency_matrix, A, K)
            new_value = objective(X)

            # If the objective improves, keep the change; otherwise, revert
            if new_value < current_value:
                current_value = new_value
            else:
                A[i] = original_value

    #print(current_value)
    return A


In [9]:
Y_t_exp_times = []

E = 350
lamada = 0.5

for times in range(10):
    print(f"Running experiment {times + 1} for ete")
    
    # Ensure `beta_star` and `t` are properly initialized before calling the function
    # Replace with actual initialization of beta_star and other parameters if needed
    Y_t_exp = while_ete_bandit(beta_star, E, lamada)  # Replace with the actual function

    # Append the results of this experiment
    Y_t_exp_times.append(Y_t_exp)

# Convert lists to NumPy arrays for easier computation
Y_t_exp_times = np.array(Y_t_exp_times)

# Save the results to .npy files
np.save(f'Y_t_exp_ete-bandit-under-interferenc-n15-k2-d267-s10_10times.npy', Y_t_exp_times)


Running experiment 1 for ete
Running experiment 2 for ete
Running experiment 3 for ete
Running experiment 4 for ete
Running experiment 5 for ete
Running experiment 6 for ete
Running experiment 7 for ete
Running experiment 8 for ete
Running experiment 9 for ete
Running experiment 10 for ete
